In [51]:
import numpy as np
import requests
import json
import time

#from matplotlib.finance import candlestick2_ohlc
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as datetime

start_time = time.time() - 299.5*60*60
resource = requests.get("https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=%s&end=9999999999&period=1800" % start_time)
data = json.loads(resource.text)

quotes = {}
quotes['open']=np.asarray([item['open'] for item in data])
quotes['close']=np.asarray([item['close'] for item in data])

time_series = quotes['open'] - quotes['close']

In [53]:
l = len(time_series) // 2
ts = time_series[l:]
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
scores = np.empty(14)
for k in range(2, 16):
    X = time_series[:l - l%k]
    X = X.reshape(l//k, k)
    KMean = KMeans(n_clusters=k)
    KMean.fit(X)
    y_true = KMean.labels_
    X1 = ts[:l - l%k]
    X1 = X1.reshape(l//k, k)
    y_pred = KMean.predict(X1)
    scores[k - 2] = accuracy_score(y_true, y_pred)
k = 2 + np.argmax(scores)
print(max(scores),k)
 

0.4533333333333333 2


In [54]:
ts = time_series
N = len(ts)
p = 2*k + 1
c = p*(p+1)//2 + p + 1 # columns
A = np.empty(shape=(c,c))
shift = N - 2*c - 1
for i in range(c):
    A[i][0] = 1
    for j in range(p):
       A[i][1 + j] = ts[shift + j]
    s = p + 1
    for j in range(p):
        for k in range(j + 1):
            A[i][s] = ts[shift + j] * ts[shift + k]
            s += 1
    shift += 1
b = np.empty(c)
shift = N - c
for i in range(c):
    b[i] = ts[shift + i]
x = np.linalg.solve(A,b)

[-4.91622645e-05 -4.04842404e-02 -1.45272807e+00 -8.21883767e-01
  1.07245316e+00  1.61653146e+00  4.07204567e+04 -1.99829717e+03
 -3.45380568e+03 -7.26742818e+04  4.03048656e+02  1.63291042e+04
 -1.13980542e+04 -7.89575232e+03 -3.63365507e+04 -3.39626774e+04
  2.48440948e+03 -1.43181457e+04  6.99682591e+03 -3.92927575e+04
 -3.19350037e+04]


In [69]:
a_0 = x[0]
a = x[1:p+1]
B = np.empty(shape=(p,p))
s = p + 1
for i in range(p):
    for j in range(i + 1):
        B[i][j] = B[j][i] = x[s]
        s += 1
t = ts[-p:]
pred = a_0 + np.dot(t, a) + np.dot(np.dot(t, B), t)
print(pred)

0.0002206627240430102
